# NLP & representation learning: Neural Embeddings, Text Classification


To use statistical classifiers with text, it is first necessary to vectorize the text. In the first practical session we explored the **Bag of Word (BoW)** model. 

Modern **state of the art** methods uses  embeddings to vectorize the text before classification in order to avoid feature engineering.

## [Dataset](https://thome.isir.upmc.fr/classes/RITAL/json_pol)


## "Modern" NLP pipeline

By opposition to the **bag of word** model, in the modern NLP pipeline everything is **embeddings**. Instead of encoding a text as a **sparse vector** of length $D$ (size of feature dictionnary) the goal is to encode the text in a meaningful dense vector of a small size $|e| <<< |D|$. 


The raw classification pipeline is then the following:

```
raw text ---|embedding table|-->  vectors --|Neural Net|--> class 
```


### Using a  language model:

How to tokenize the text and extract a feature dictionnary is still a manual task. To directly have meaningful embeddings, it is common to use a pre-trained language model such as `word2vec` which we explore in this practical.

In this setting, the pipeline becomes the following:
```
      
raw text ---|(pre-trained) Language Model|--> vectors --|classifier (or fine-tuning)|--> class 
```


- #### Classic word embeddings

 - [Word2Vec](https://arxiv.org/abs/1301.3781)
 - [Glove](https://nlp.stanford.edu/projects/glove/)


- #### bleeding edge language models techniques (see next)

 - [UMLFIT](https://arxiv.org/abs/1801.06146)
 - [ELMO](https://arxiv.org/abs/1802.05365)
 - [GPT](https://blog.openai.com/language-unsupervised/)
 - [BERT](https://arxiv.org/abs/1810.04805)






### Goal of this session:

1. Train word embeddings on training dataset
2. Tinker with the learnt embeddings and see learnt relations
3. Tinker with pre-trained embeddings.
4. Use those embeddings for classification
5. Compare different embedding models
6. Pytorch first look: learn to generate text.

## STEP 0: Loading data 

In [1]:
import json
from collections import Counter

# Loading json
with open("json_pol",encoding="utf-8") as f:
    data = f.readlines()
    json_data = json.loads(data[0])
    train = json_data["train"]
    test = json_data["test"]
    

# Quick Check
counter_train = Counter((x[1] for x in train))
counter_test = Counter((x[1] for x in test))
print("Number of train reviews : ", len(train))
print("----> # of positive : ", counter_train[1])
print("----> # of negative : ", counter_train[0])
print("")
print(train[0])
print("")
print("Number of test reviews : ",len(test))
print("----> # of positive : ", counter_test[1])
print("----> # of negative : ", counter_test[0])

print("")
print(test[0])
print("")

Number of train reviews :  25000
----> # of positive :  12500
----> # of negative :  12500

["The undoubted highlight of this movie is Peter O'Toole's performance. In turn wildly comical and terribly terribly tragic. Does anybody do it better than O'Toole? I don't think so. What a great face that man has!<br /><br />The story is an odd one and quite disturbing and emotionally intense in parts (especially toward the end) but it is also oddly touching and does succeed on many levels. However, I felt the film basically revolved around Peter O'Toole's luminous performance and I'm sure I wouldn't have enjoyed it even half as much if he hadn't been in it.", 1]

Number of test reviews :  25000
----> # of positive :  12500
----> # of negative :  12500

['Although credit should have been given to Dr. Seuess for stealing the story-line of "Horton Hatches The Egg", this was a fine film. It touched both the emotions and the intellect. Due especially to the incredible performance of seven year old 

## Word2Vec: Quick Recap

**[Word2Vec](https://arxiv.org/abs/1301.3781) is composed of two distinct language models (CBOW and SG), optimized to quickly learn word vectors**


given a random text: `i'm taking the dog out for a walk`



### (a) Continuous Bag of Word (CBOW)
    -  predicts a word given a context
    
maximizing `p(dog | i'm taking the ___ out for a walk)`
    
### (b) Skip-Gram (SG)               
    -  predicts a context given a word
    
 maximizing `p(i'm taking the out for a walk | dog)`



   

## STEP 1: train a language model (word2vec)

Gensim has one of [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) fastest implementation.


### Train:

In [2]:
# if gensim not installed yet
# ! pip install gensim

In [3]:
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

text = [t.split() for t,p in train]

# the following configuration is the default configuration
w2v = gensim.models.word2vec.Word2Vec(sentences=text,
                                vector_size=100, window=5,               ### here we train a cbow model 
                                min_count=5,                      
                                sample=0.001, workers=3,
                                sg=1, hs=0, negative=5,        ### set sg to 1 to train a sg model
                                cbow_mean=1, epochs=5)

2023-03-19 13:32:02,221 : INFO : collecting all words and their counts
2023-03-19 13:32:02,222 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-03-19 13:32:03,116 : INFO : PROGRESS: at sentence #10000, processed 2358544 words, keeping 155393 word types
2023-03-19 13:32:03,936 : INFO : PROGRESS: at sentence #20000, processed 4675912 words, keeping 243050 word types
2023-03-19 13:32:04,341 : INFO : collected 280617 word types from a corpus of 5844680 raw words and 25000 sentences
2023-03-19 13:32:04,342 : INFO : Creating a fresh vocabulary
2023-03-19 13:32:04,603 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 49345 unique words (17.58% of original 280617, drops 231272)', 'datetime': '2023-03-19T13:32:04.603310', 'gensim': '4.3.1', 'python': '3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'prepare_vocab'}
2023-03-19 13:32:04,604 : INFO : Word2Vec lifecycle event {

In [9]:
# Worth it to save the previous embedding
w2v.save("W2v-movies.dat")
# You will be able to reload them:
# w2v = gensim.models.Word2Vec.load("W2v-movies.dat")
# and you can continue the learning process if needed

2023-03-18 18:10:18,128 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'W2v-movies.dat', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-03-18T18:10:18.128194', 'gensim': '4.3.1', 'python': '3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'saving'}
2023-03-18 18:10:18,129 : INFO : not storing attribute cum_table
2023-03-18 18:10:18,541 : INFO : saved W2v-movies.dat


## STEP 2: Test learnt embeddings

The word embedding space directly encodes similarities between words: the vector coding for the word "great" will be closer to the vector coding for "good" than to the one coding for "bad". Generally, [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) is the distance used when considering distance between vectors.

KeyedVectors have a built in [similarity](https://radimrehurek.com/gensim/models /keyedvectors.html#gensim.models.keyedvectors.BaseKeyedVectors.similarity) method to compute the cosine similarity between words

In [10]:
# is great really closer to good than to bad ?
print("great and good:",w2v.wv.similarity("great","good"))
print("great and bad:",w2v.wv.similarity("great","bad"))

great and good: 0.76896423
great and bad: 0.46418405


Since cosine distance encodes similarity, neighboring words are supposed to be similar. The [most_similar](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.BaseKeyedVectors.most_similar) method returns the `topn` words given a query.

In [11]:
# The query can be as simple as a word, such as "movie"

# Try changing the word
w2v.wv.most_similar("movie",topn=5) # 5 most similar words
#w2v.wv.most_similar("awesome",topn=5)
#w2v.wv.most_similar("actor",topn=5)

[('film', 0.9401601552963257),
 ('"film"', 0.8285843133926392),
 ('"movie"', 0.7925335764884949),
 ('programme', 0.7681950926780701),
 ('movie,', 0.7639946937561035)]

But it can be a more complicated query
Word embedding spaces tend to encode much more.

The most famous exemple is: `vec(king) - vec(man) + vec(woman) => vec(queen)`

In [12]:
# What is awesome - good + bad ?
w2v.wv.most_similar(positive=["awesome","bad"],negative=["good"],topn=3)  

w2v.wv.most_similar(positive=["actor","woman"],negative=["man"],topn=3) # do the famous exemple works for actor ?


# Try other things like plurals for exemple.

[('actress', 0.8198764324188232),
 ('actress,', 0.7319310307502747),
 ('actress.', 0.6946509480476379)]

To test learnt "synctactic" and "semantic" similarities, Mikolov et al. introduced a special dataset containing a wide variety of three way similarities.

In [13]:
out = w2v.wv.evaluate_word_analogies("questions-words.txt",case_insensitive=True)  #original semantic syntactic dataset.

2023-03-18 18:11:33,875 : INFO : Evaluating word analogies for top 300000 words in the model on questions-words.txt
2023-03-18 18:11:34,628 : INFO : capital-common-countries: 1.3% (2/156)
2023-03-18 18:11:34,969 : INFO : capital-world: 1.8% (2/111)
2023-03-18 18:11:35,030 : INFO : currency: 0.0% (0/18)
2023-03-18 18:11:35,909 : INFO : city-in-state: 0.0% (0/301)
2023-03-18 18:11:37,064 : INFO : family: 31.9% (134/420)
2023-03-18 18:11:39,285 : INFO : gram1-adjective-to-adverb: 2.0% (17/870)
2023-03-18 18:11:40,665 : INFO : gram2-opposite: 3.3% (18/552)
2023-03-18 18:11:44,485 : INFO : gram3-comparative: 17.9% (213/1190)
2023-03-18 18:11:46,561 : INFO : gram4-superlative: 9.4% (71/756)
2023-03-18 18:11:48,656 : INFO : gram5-present-participle: 19.3% (157/812)
2023-03-18 18:11:51,200 : INFO : gram6-nationality-adjective: 1.2% (12/967)
2023-03-18 18:11:54,639 : INFO : gram7-past-tense: 21.3% (268/1260)
2023-03-18 18:11:56,830 : INFO : gram8-plural: 6.9% (56/812)
2023-03-18 18:11:58,679 : 

**When training the w2v models on the review dataset, since it hasn't been learnt with a lot of data, it does not perform very well.**


## STEP 3: Loading a pre-trained model

In Gensim, embeddings are loaded and can be used via the ["KeyedVectors"](https://radimrehurek.com/gensim/models/keyedvectors.html) class

> Since trained word vectors are independent from the way they were trained (Word2Vec, FastText, WordRank, VarEmbed etc), they can be represented by a standalone structure, as implemented in this module.

>The structure is called “KeyedVectors” and is essentially a mapping between entities and vectors. Each entity is identified by its string id, so this is a mapping between {str => 1D numpy array}.

>The entity typically corresponds to a word (so the mapping maps words to 1D vectors), but for some models, they key can also correspond to a document, a graph node etc. To generalize over different use-cases, this module calls the keys entities. Each entity is always represented by its string id, no matter whether the entity is a word, a document or a graph node.

In [ ]:
from gensim.test.utils import get_tmpfile
import gensim.downloader as api
bload = True
fname = "word2vec-google-news-300"
sdir = ""

if(bload==True):
    wv_pre_trained = gensim.models.KeyedVectors.load(sdir+fname+".dat")
else:    
    wv_pre_trained = api.load(fname)
    wv_pre_trained.save(sdir+fname+".dat")

out = wv_pre_trained.evaluate_word_analogies("questions-words.txt",case_insensitive=True)  #original semantic syntactic dataset.

# It performs much better than not pre-trained model because this model has been pre-trained with a lot of data. We find an accuracy of 74%.
print(wv_pre_trained, "\n\n", out)


**Perform the "synctactic" and "semantic" evaluations again. Conclude on the pre-trained embeddings.**

## STEP 4:  sentiment classification

In the previous practical session, we used a bag of word approach to transform text into vectors.
Here, we propose to try to use word vectors (previously learnt or loaded).


### <font color='green'> Since we have only word vectors and that sentences are made of multiple words, we need to aggregate them. </font>


### (1) Vectorize reviews using word vectors:

Word aggregation can be done in different ways:

- Sum
- Average
- Min/feature
- Max/feature

#### a few pointers:

- `w2v.wv.vocab` is a `set()` of the vocabulary (all existing words in your model)
- `np.minimum(a,b) and np.maximum(a,b)` respectively return element-wise min/max 

In [6]:
import numpy as np
# We first need to vectorize text:
# First we propose to a sum of them


def vectorize(text,mean=False):
    """
    This function should vectorize one review

    input: str
    output: np.array(float)
    """    
    vec = []
    for word in text.split():
        try:
            vec.append(w2v.wv[word])
        except KeyError:
            continue
    return np.mean(np.array(vec), axis=0)


classes = [pol for text,pol in train]
X = [vectorize(text) for text,pol in train]
X_test = [vectorize(text) for text,pol in test]
true = [pol for text,pol in test]

#let's see what a review vector looks like.
print(X[0])

[-0.07262014  0.17286564 -0.07118223  0.001789    0.01288627 -0.23364043
  0.13452812  0.41324234 -0.30692497 -0.139145   -0.03348449 -0.33013242
  0.00138811  0.09071711  0.06381789 -0.15111387  0.16269377 -0.12357354
 -0.05258307 -0.3644595   0.13033417  0.08568899  0.08963778 -0.1406589
  0.01863531 -0.03317967 -0.28280744 -0.00393722 -0.1345734   0.19159377
  0.2949677  -0.13948822  0.0833481  -0.34438506  0.01773646  0.19819117
 -0.03365245  0.19119875 -0.09158282 -0.13051619  0.14398322 -0.09799657
 -0.07274494  0.05602719  0.10379355  0.00375845 -0.11315077 -0.14127155
  0.22803544  0.17338903  0.08904986 -0.35945016 -0.05061587  0.01823387
 -0.08273222  0.09224615  0.07339675 -0.02819855 -0.27725074  0.02276383
 -0.01606501 -0.08772795  0.24104749  0.05665727 -0.17684427  0.24432835
 -0.00482281  0.13183242 -0.31785557  0.08800955  0.01735726  0.1341805
  0.19851846  0.01587687  0.1447176  -0.02213734  0.07950871 -0.07664558
 -0.08680249  0.06741351 -0.24302265 -0.08241361 -0.2

### (2) Train a classifier 
as in the previous practical session, train a logistic regression to do sentiment classification with word vectors



In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Scikit Logistic Regression
    
lr = LogisticRegression(random_state=0)
lr.fit(X, classes)
print(lr.score(X_test, true))


0.82048


performance should be worst than with bag of word (~80%). Sum/Mean aggregation does not work well on long reviews (especially with many frequent words). This adds a lot of noise.

## **Todo** :  Try answering the following questions:

- Which word2vec model works best: skip-gram or cbow
- Do pretrained vectors work best than those learnt on the train dataset ?



**(Bonus)** To have a better accuracy, we could try two things:
- Better aggregation methods (weight by tf-idf ?)
- Another word vectorizing method such as [fasttext](https://radimrehurek.com/gensim/models/fasttext.html)
- A document vectorizing method such as [Doc2Vec](https://radimrehurek.com/gensim/models/doc2vec.html)